In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
%matplotlib inline

import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression


# Regression Analysis: Seasonal Effects with Sklearn Linear Regression
In this notebook, you will build a SKLearn linear regression model to predict Yen futures ("settle") returns with *lagged* Yen futures returns. 

In [2]:
# Futures contract on the Yen-dollar exchange rate:
# This is the continuous chain of the futures contracts that are 1 month to expiration
yen_futures = pd.read_csv(
    Path("yen.csv"), index_col="Date", infer_datetime_format=True, parse_dates=True
)
yen_futures.head()

Open    High     Low    Last  Change  Settle  Volume  \
Date                                                                 
1976-08-02  3398.0  3401.0  3398.0  3401.0     NaN  3401.0     2.0   
1976-08-03  3401.0  3401.0  3401.0  3401.0     NaN  3401.0     0.0   
1976-08-04  3401.0  3401.0  3401.0  3401.0     NaN  3401.0     0.0   
1976-08-05  3401.0  3401.0  3401.0  3401.0     NaN  3401.0     0.0   
1976-08-06  3401.0  3401.0  3401.0  3401.0     NaN  3401.0     0.0   

            Previous Day Open Interest  
Date                                    
1976-08-02                         1.0  
1976-08-03                         1.0  
1976-08-04                         1.0  
1976-08-05                         1.0  
1976-08-06                         1.0

In [3]:
# Trim the dataset to begin on January 1st, 1990
yen_futures = yen_futures.loc["1990-01-01":, :]
yen_futures.head()

Open    High     Low    Last  Change  Settle   Volume  \
Date                                                                  
1990-01-02  6954.0  6954.0  6835.0  6847.0     NaN  6847.0  48336.0   
1990-01-03  6877.0  6910.0  6865.0  6887.0     NaN  6887.0  38206.0   
1990-01-04  6937.0  7030.0  6924.0  7008.0     NaN  7008.0  49649.0   
1990-01-05  6952.0  6985.0  6942.0  6950.0     NaN  6950.0  29944.0   
1990-01-08  6936.0  6972.0  6936.0  6959.0     NaN  6959.0  19763.0   

            Previous Day Open Interest  
Date                                    
1990-01-02                     51473.0  
1990-01-03                     53860.0  
1990-01-04                     55699.0  
1990-01-05                     53111.0  
1990-01-08                     52072.0

# Data Preparation

### Returns

In [4]:
# Create a series using "Settle" price percentage returns, drop any nan"s, and check the results:
# (Make sure to multiply the pct_change() results by 100)
# In this case, you may have to replace inf, -inf values with np.nan"s

returns = (yen_futures[["Settle"]].pct_change() * 100)
returns = returns.replace(-np.inf, np.nan).dropna()
returns.tail()

Settle
Date                
2019-10-09 -0.410601
2019-10-10 -0.369458
2019-10-11 -0.564304
2019-10-14  0.151335
2019-10-15 -0.469509

### Lagged Returns 

In [5]:
# Create a lagged return using the shift function

returns['Settle'] = returns.copy()
returns['Lagged_Return'] = returns.shift()
# Drop NaN values
returns = returns.dropna()
returns

Settle  Lagged_Return
Date                               
1990-01-04  1.756933       0.584197
1990-01-05 -0.827626       1.756933
1990-01-08  0.129496      -0.827626
1990-01-09 -0.632275       0.129496
1990-01-10 -0.404917      -0.632275
1990-01-11  0.087121      -0.404917
1990-01-12 -0.290149       0.087121
1990-01-15  0.349192      -0.290149
1990-01-16  0.000000       0.349192
1990-01-17  0.000000       0.000000
1990-01-18 -0.637959       0.000000
1990-01-19  0.321027      -0.637959
1990-01-22 -0.480000       0.321027
1990-01-23  0.380006      -0.480000
1990-01-24  0.232964       0.380006
1990-01-25  1.002324       0.232964
1990-01-26  0.992377       1.002324
1990-01-29 -0.270578       0.992377
1990-01-30 -1.342282      -0.270578
1990-01-31  0.434216      -1.342282
1990-02-01 -0.360282       0.434216
1990-02-02 -0.433902      -0.360282
1990-02-05  0.493899      -0.433902
1990-02-06 -0.390286       0.493899
1990-02-07 -0.145117      -0.390286
1990-02-08  0.130795      -0.145117
1990-02-09  0.391872       0.130795
1990-02-12  0.303600       0.391872
1990-02-13 -0.014413       0.303600
1990-02-14  0.028831      -0.014413
...              ...            ...
2019-09-04 -0.354892       0.196370
2019-09-05 -0.547523      -0.354892
2019-09-06  0.064140      -0.547523
2019-09-09 -0.261738       0.064140
2019-09-10 -0.257069      -0.261738
2019-09-11 -0.327534      -0.257069
2019-09-12 -0.377094      -0.327534
2019-09-13  0.000000      -0.377094
2019-09-16  0.173038       0.000000
2019-09-17  0.404858       0.173038
2019-09-18 -0.145161       0.404858
2019-09-19  0.333818      -0.145161
2019-09-20  0.246847       0.333818
2019-09-23  0.192709       0.246847
2019-09-24  0.395363       0.192709
2019-09-25 -0.739716       0.395363
2019-09-26  0.000000      -0.739716
2019-09-27 -0.058975       0.000000
2019-09-30 -0.241403      -0.058975
2019-10-01  0.328028      -0.241403
2019-10-02  0.482393       0.328028
2019-10-03  0.256041       0.482393
2019-10-04  0.021282       0.256041
2019-10-07 -0.414916       0.021282
2019-10-08  0.170931      -0.414916
2019-10-09 -0.410601       0.170931
2019-10-10 -0.369458      -0.410601
2019-10-11 -0.564304      -0.369458
2019-10-14  0.151335      -0.564304
2019-10-15 -0.469509       0.151335

[7513 rows x 2 columns]

### Train Test Split

In [8]:
# Create a train/test split for the data using 2018-2019 for testing and the rest for training
train = yen_futures.loc[:'2017']
test = yen_futures.loc['2018':,:]
train.head()

Open    High     Low    Last  Change  Settle   Volume  \
Date                                                                  
1990-01-02  6954.0  6954.0  6835.0  6847.0     NaN  6847.0  48336.0   
1990-01-03  6877.0  6910.0  6865.0  6887.0     NaN  6887.0  38206.0   
1990-01-04  6937.0  7030.0  6924.0  7008.0     NaN  7008.0  49649.0   
1990-01-05  6952.0  6985.0  6942.0  6950.0     NaN  6950.0  29944.0   
1990-01-08  6936.0  6972.0  6936.0  6959.0     NaN  6959.0  19763.0   

            Previous Day Open Interest  
Date                                    
1990-01-02                     51473.0  
1990-01-03                     53860.0  
1990-01-04                     55699.0  
1990-01-05                     53111.0  
1990-01-08                     52072.0

In [ ]:
# Create four dataframes:
# X_train (training set using just the independent variables), X_test (test set of of just the independent variables)
# Y_train (training set using just the "y" variable, i.e., "Futures Return"), Y_test (test set of just the "y" variable):
# YOUR CODE HERE!

In [ ]:
X_train

# Linear Regression Model

In [ ]:
# Create a Linear Regression model and fit it to the training data
from sklearn.linear_model import LinearRegression

# Fit a SKLearn linear regression using just the training set (X_train, Y_train):
# YOUR CODE HERE!

# Make predictions using the Testing Data

Note: We want to evaluate the model using data that it has never seen before, in this case: X_test.

In [ ]:
# Make a prediction of "y" values using just the test dataset
# YOUR CODE HERE!

In [ ]:
# Assemble actual y data (Y_test) with predicted y data (from just above) into two columns in a dataframe:
# YOUR CODE HERE!

In [ ]:
# Plot the first 20 predictions vs the true values
# YOUR CODE HERE!

# Out-of-Sample Performance

Evaluate the model using "out-of-sample" data (X_test and y_test)

In [ ]:
from sklearn.metrics import mean_squared_error
# Calculate the mean_squared_error (MSE) on actual versus predicted test "y" 
# YOUR CODE HERE!

# Using that mean-squared-error, calculate the root-mean-squared error (RMSE):
# YOUR CODE HERE!

# In-Sample Performance

Evaluate the model using in-sample data (X_train and y_train)

In [ ]:
# Construct a dataframe using just the "y" training data:
# YOUR CODE HERE!

# Add a column of "in-sample" predictions to that dataframe:  
# YOUR CODE HERE!

# Calculate in-sample mean_squared_error (for comparison to out-of-sample)
# YOUR CODE HERE!

# Calculate in-sample root mean_squared_error (for comparison to out-of-sample)
# YOUR CODE HERE!

# Conclusions

YOUR CONCLUSIONS HERE!